# Dear Spencer,

I really enjoyed speaking with you in Austin about the existential threat of serial killers, and I completely agree that people are not taking this seriously enough.  

I decided to do a little big of digging to see what kind of threat we face.  There is a faily well known dataset on [Kaggle](https://www.kaggle.com/datasets/vesuvius13/serial-killers-dataset) that many people will use to make observations about serial killers.  However, that data is 3 years old.  Instead I decided to scrape the data from [This Wiki Page](https://en.wikipedia.org/wiki/List_of_serial_killers_by_number_of_victims).  It appears to be the same dataset, just more up to date.  By scraping the data, I have additional 500+ rows to work with.

I've included all of my code as well as the `requirements.txt` file in this repo, so feel free to expirement.  Together, we will get to the bottom of this.

In [41]:
import pandas as pd
import re
import numpy as np
import requests
import time

# Functions

In [42]:
def extract_largest_number(possible_victims):
    """
    Extracts the largest number from a given input string representing possible victim counts,
    converts non-string numeric inputs to integers, and handles NaN values.

    Parameters:
    ----------
    possible_victims : str, int, float, or NaN
        The input representing possible victim counts. It can be a string containing numbers (e.g., '300+', '194-300+'),
        a single number, or a NaN.

    Returns:
    -------
    int
        The largest number found in the input string if it contains multiple numbers, 
        the integer representation if the input is a single number, or 0 if the input is NaN or no valid number is found.
    
    """
    if pd.isna(possible_victims):
        return 0  # Convert NaN to 0
    elif isinstance(possible_victims, str):
        # Find all numbers in the string
        numbers = re.findall(r'\d+', possible_victims)
        # Convert numbers to integers
        numbers = list(map(int, numbers))
        # Return the largest number
        return max(numbers) if numbers else 0
    elif isinstance(possible_victims, (int, float)):
        return int(possible_victims)
    else:
        return 0
    
def get_lat_lon(country_name):
    url = f"https://restcountries.com/v3.1/name/{country_name}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data:
                # Get the latitude and longitude from the API response
                lat, lon = data[0]['latlng']
                return lat, lon
        return None, None
    except Exception as e:
        print(f"Error fetching data for {country_name}: {e}")
        return None, None

# Read in Data

In [43]:
df = pd.read_csv(r'data/serial_killers_wiki.csv')
display(df.head(25))

,Name,Country,Years active,Proven victims,Possible victims,Notes
0,Luis Garavito,Colombia Ecuador[3] Venezuela[3],1992–1999[3],193,194–300+,"Child-murderer, torture-killer, and rapist kno..."
1,Pedro López,Colombia Peru Ecuador,1969–1980,110,300+,"Child-murderer and rapist, known as ""The Monst..."
2,Javed Iqbal,Pakistan,1998–1999,100,100,"Child-murderer and rapist, known as ""Kukri"", I..."
3,Mikhail Popkov,Russia,1992–2010,83,86+,"Serial rapist-killer nicknamed ""The Werewolf"",..."
4,Daniel Camargo Barbosa,Colombia Ecuador Brazil (alleged)[14],1974–1986,72,180[14],"Child and woman murderer, believed to have pos..."
5,Pedro Rodrigues Filho,Brazil,1967–2003,71,100+,Rodrigues Filho claimed to have killed over 10...
6,Kampatimar Shankariya,India,1977–1978,70,70+,Shankariya was an Indian serial killer who use...
7,Hoshang Amini,Iran,1954–1962,67,67,"Known as the ""Ghost of the Qanat Wells"". Murde..."
8,Yang Xinhai,China,1999–2003,67,67,"Known as the ""Monster Killer"". Yang would ente..."
9,Abul Djabar,Afghanistan,1970 and earlier,65,300+,Killed 65 men and boys by strangling them with...


# Cleaning Data
1. Location
    * For the sake of simplicity, I'm going to select the first country listed for each serial killer.
2. Years Active
    * I'm going to split these up into `start` and `end` as best I can.
    * Some of these are listed as `Year and earlier` so I've asked chatgpt to guess at what year those people got started, and I've updated the dataset.
    * Create a duration column to indicate how long someone was active. 
3. Possible Victims
    * I'm going to take the alrgest number and remove the `+` sign so that we can treat these as numeric values.

## Location

In [44]:
# Remove all but the first country
df['Country'] =  df['Country'].apply(lambda x: re.split(r'[\xa0\[]', x)[0].strip())

df.head(25)

,Name,Country,Years active,Proven victims,Possible victims,Notes
0,Luis Garavito,Colombia,1992–1999[3],193,194–300+,"Child-murderer, torture-killer, and rapist kno..."
1,Pedro López,Colombia,1969–1980,110,300+,"Child-murderer and rapist, known as ""The Monst..."
2,Javed Iqbal,Pakistan,1998–1999,100,100,"Child-murderer and rapist, known as ""Kukri"", I..."
3,Mikhail Popkov,Russia,1992–2010,83,86+,"Serial rapist-killer nicknamed ""The Werewolf"",..."
4,Daniel Camargo Barbosa,Colombia,1974–1986,72,180[14],"Child and woman murderer, believed to have pos..."
5,Pedro Rodrigues Filho,Brazil,1967–2003,71,100+,Rodrigues Filho claimed to have killed over 10...
6,Kampatimar Shankariya,India,1977–1978,70,70+,Shankariya was an Indian serial killer who use...
7,Hoshang Amini,Iran,1954–1962,67,67,"Known as the ""Ghost of the Qanat Wells"". Murde..."
8,Yang Xinhai,China,1999–2003,67,67,"Known as the ""Monster Killer"". Yang would ente..."
9,Abul Djabar,Afghanistan,1970 and earlier,65,300+,Killed 65 men and boys by strangling them with...


## Years Active
The `Years active` column comes in varying formats.  I asked chatgpt to create a dictionary of better dates for any of the rows that are not formatted as `YYYY-YYYY`.  This may not be perfect, but it will be good enough for this analsysis.

In [45]:
# earlier_df = df[df['Years active'].str.contains('and', na=False)].reset_index()

# print(f'There are {len(earlier_df)} instances where the Years active column contains "and"')

# display(earlier_df)

In [46]:
# Mapping dictionary for updated "Years active" ranges -- FROM CHATGPT
years_active_mapping = {
    'Abul Djabar': '1965-1970',
    'Hadj Mohammed Mesfewi': '1901-1906',
    'Darren Deon Vann': '2009-2014',
    'William Dathan Holbert': '2005-2010',
    'Ernesto Picchioni': '1944-1949',
    'Robert Hicks Murray': '1907-1912',
    'Akku Yadav': '1999-2004',
    'Arnfinn Nesset': '1978-1983',
    'Christine Malèvre': '1993-1998',
    'Edson Izidoro Guimarães': '1994-1999',
    'The Skin Hunters': '1997-2002',
    'Daisuke Mori': '1995-2000',
    'Thomas Henry McMonigle': '1940-1945',
    'Monster of the Mangones': '1963-1970',
    'Milton Sipalo': '1980-1980',
    'Juan Corona': '1971-1971',
    'Abdullah Shah': '1990-1999',
    'Velaphi Ndlangamandla': '1998-1998',
    'Paul John Knowles': '1974-1974',
    'Asande Baninzi': '2001-2001',
    'Darbara Singh': '2004-2004',
    'Chen Fuzhao': '2003-2003',
    'Juana Barraza': '1998-2006',
    'Elias Xitavhudzi': '1950-1959',
    'Jimmy Maketta': '2005-2005',
    'Atlanta Ripper': '1911-1911',
    'Florencio Fernández': '1950-1959',
    'Marcelo Costa de Andrade': '1991-1991',
    'Amir Qayyum': '2005-2005',
    'Denis Kazungu': '2023-2023',
    'Kaspars Petrovs': '2003-2003',
    'Sleepy Hollow Killer': '1990-2007',
    'Thozamile Taki': '2007-2007',
    'Enriqueta Martí': '1900-1912',
    'Nikolai Shestakov': '1975-1975',
    '"Paraquat murders" killer': '1985-1985',
    'Robledo Puch': '1971-1971',
    'Roshu Kha': '2008-2008',
    'Martin Lecián': '1927-1927',
    'Milton Johnson': '1983-1983',
    'Sergey Cherny': '1999-1999',
    'Bobby Joe Long': '1984-1984',
    'Ali Kaya': '1997-2014',
    'Chen Yongfeng': '2003-2003',
    'Edgecombe County Serial Killer': '2000-2009',
    'Timothy Krajcir': '2000-2009',
    'Christopher Wilder': '1984-1984',
    'Kiyoshi Okubo': '1971-1971',
    'Ivan Milat': '1990-1999',
    'Kenneth Erskine': '1986-1986',
    'Vladimir Kuzmin': '1997-1997',
    'Derrick Todd Lee': '2002-2002',
    'Doug Clark': '1980-1980',
    'Harrison Graham': '1987-1987',
    'Paul Dennis Reid': '1997-1997',
    'Sibusiso Duma': '2007-2007',
    'Władysław Mazurkiewicz': '1950-1959',
    'Cristopher Chávez Cuellar': '1990-2015',
    'Alfredo Galán': '2003-2003',
    'Eugen Weidmann': '1937-1937',
    'Cleophus Prince Jr.': '1990-1990',
    'Gary Ray Bowles': '1994-1994',
    'Nicholas Lungisa Ncama': '1997-1997',
    'Steve Wright': '2006-2006',
    'Allan Joseph Legere': '1989-1989',
    'Sek Kim Wah': '1983-1983',
    'Phantom Killer': '1946-1946',
    'San Mateo slasher': '1976-1976',
    'Harry Powers': '1931-1931',
    'South Dade Killer': '1975-1975',
    'Archibald Hall': '1977-1977',
    'Colin Ireland': '1993-1993',
    'Romulus Vereş': '1970-1979',
    'Akira Nishiguchi': '1963-1963',
    'Anatoly Biryukov': '1977-1977',
    'Angus Sinclair': '1961-1982',
    'Cary Stayner': '1999-1999',
    'Christopher Peterson': '1990-1990',
    'Dallen Bounds': '1999-1999',
    'Heinrich Pommerenke': '1959-1959',
    'Howell Donaldson III': '2017-2017',
    'Cayetano Santos Godino': '1912-1912',
    'Eastbound Strangler': '2006-2006',
    'Gordon Cummins': '1942-1942',
    'Marc Sappington': '2001-2001',
    'Léopold Dion': '1963-1963',
    'Lam Kor-wan': '1980-1989',
    'Michael Lupo': '1986-1986',
    'James Swann': '1993-1993',
    'Alexander Tchayka': '1994-1994',
    'Peter Moore': '1995-1995',
    'Hiroaki Hidaka': '1996-1996',
    'Nicolai Bonner': '2005-2005',
    'Raúl Osiel Marroquín': '2005-2005',
    'Dale Cregan': '2012-2012',
    'Pedro Padilla Flores': '1986-1986',
    'Stephen Morin': '1981-1981',
    'Sidney Cooke': '1960-1989',
    'Ronald Janssen': '1990-2010',
    'Anthony Hardy': '2002-2002',
    'Hans van Zon': '1967-1967',
    'Paul Denyer': '1993-1993',
    'Gordon Northcott': '1928-1928',
    'Özkan Zengin': '2008-2008',
    'Khalil Wheeler-Weaver': '2016-2016',
    'Westley Allan Dodd': '1989-1989',
    'John Martin': '1995-1995',
    'Leon Dorsey': '1994-1994',
    'Eddie Leonski': '1942-1942',
    'Lam Kwok-wai': '1993-1993',
    'Dana Sue Gray': '1994-1994',
    'Robin Ligus': '1994-1994',
    'Matthew James Harris': '1998-1998',
    'Hiroshi Maeue': '2005-2005',
    'Özgür Dengiz': '2007-2007',
    'Joanna Dennehy': '2013-2013',
    'Alejandro Máynez': '1980-1999',
    'José Luis Calva': '2007-2007',
    'John Norman Collins': '1969-1969',
    'Baba Anujka': '1890-1928',
    'Kermit Gosnell': '1989-2010',
    'Ann Arbor Hospital Killer': '1975-1975',
    'Petr Zelenka': '2006-2006',
    'Kimberly Clark Saenz': '2008-2008',
    'Richard Angelo': '1987-1987',
    'Colin Norris': '2002-2002',
    'Beverley Allitt': '1991-1991',
    'Abraao Jose Bueno': '2005-2005',
    'Victorino Chua': '2011-2011',
    'Murder Incorporated': '1920-1940',
    'Abboud and Khajawa': '1900-1917',
    'Ivanova and Olga Tamarin': '1909-1909',
    'Viktor Sayenko and Igor Suprunyuk': '2007-2007',
    'Marcelo de Jesus Silva and his death squad': '2000-2010',
    'Wang Zongfang and Wang Zongwei': '1983-1983',
    'John Allen Muhammad and Lee Boyd Malvo': '2002-2002',
    'José Miculax Bux and Mariano Macú Miculax': '1946-1946',
    'Yevgeny Nagorny and Sergei Stavitsky': '1998-1998',
    'The Hernández Brothers Sect': '1963-1963',
    'Rudolfo Infante and Anna Villeda': '1991-1991',
    'Dale Hausner and Samuel Dieteman': '2006-2006',
    'Johannes van Rooyen and Dumisani Makhubela': '2005-2005',
    'Hermann Duft and Hans Wilhelm Bassenauer': '1969-1969',
    'Lawrence Bittaker and Roy Norris': '1979-1979',
    'James Gregory Marlow and Cynthia Coffman': '1986-1986',
    'Gwendolyn Graham and Catherine May Wood': '1987-1987',
    'Chijon family': '1993-1993',
    'David and Catherine Birnie': '1986-1986',
    'John Duffy and David Mulcahy': '1985-1986',
    '"Highway of Tears" Killer': '1980-present',
    'Orlando Sabino': '1966-1971',
    'Kieran Patrick Kelly': '1953-1983',
    'Billy Edwin Reid': '1989-1989',
    'Clementine Barnabet': '1911-1911',
    '"Texas Killing Fields" Killers': '1970-2000',
    'Jessie McTavish': '1974-1974',
    'Abdul Latif Sharif': '1995-1995',
    'Lowell Amos': '1979-1994',
    'Bennett Clark Hyde': '1909-1909',
    'David Moor': '1970-1999',
    'Ivomoku Bakusuba': '1940-1949',
    'Manchester Pusher': '2007-present',
    'Smiley Face Killer': '1990-2000',
    'Dr. X killings': '1966-1966'
}

# Update the "Years active" column with the correct values
df['Years active'] = df.apply(lambda row: years_active_mapping[row['Name']] if row['Name'] in years_active_mapping else row['Years active'], axis=1)

df.head(25)


,Name,Country,Years active,Proven victims,Possible victims,Notes
0,Luis Garavito,Colombia,1992–1999[3],193,194–300+,"Child-murderer, torture-killer, and rapist kno..."
1,Pedro López,Colombia,1969–1980,110,300+,"Child-murderer and rapist, known as ""The Monst..."
2,Javed Iqbal,Pakistan,1998–1999,100,100,"Child-murderer and rapist, known as ""Kukri"", I..."
3,Mikhail Popkov,Russia,1992–2010,83,86+,"Serial rapist-killer nicknamed ""The Werewolf"",..."
4,Daniel Camargo Barbosa,Colombia,1974–1986,72,180[14],"Child and woman murderer, believed to have pos..."
5,Pedro Rodrigues Filho,Brazil,1967–2003,71,100+,Rodrigues Filho claimed to have killed over 10...
6,Kampatimar Shankariya,India,1977–1978,70,70+,Shankariya was an Indian serial killer who use...
7,Hoshang Amini,Iran,1954–1962,67,67,"Known as the ""Ghost of the Qanat Wells"". Murde..."
8,Yang Xinhai,China,1999–2003,67,67,"Known as the ""Monster Killer"". Yang would ente..."
9,Abul Djabar,Afghanistan,1965-1970,65,300+,Killed 65 men and boys by strangling them with...


## Split years active into start and end
- Note: Copy the `–` from the data, as it is not the same as `-` on the keyboard.

In [47]:
# remove brackets
df['Years active'] = df['Years active'].str.replace(r'\[\d+\]', '', regex=True)

# replace weird hiphen with standard hiphen 
df['Years active'] = df['Years active'].str.replace('–', '-')

# split the years active into start and end columns
df[['Start', 'End']] = df['Years active'].str.split('-', expand=True)

# remove the years active column
df.drop('Years active', axis=1, inplace=True)

# Change "preset" to 2024 in the "End" column
df['End'].replace('present', '2024', inplace=True)


display(df.head(25))

C:\Users\Ryan\AppData\Local\Temp\ipykernel_23740\127901694.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['End'].replace('present', '2024', inplace=True)


,Name,Country,Proven victims,Possible victims,Notes,Start,End
0,Luis Garavito,Colombia,193,194–300+,"Child-murderer, torture-killer, and rapist kno...",1992,1999
1,Pedro López,Colombia,110,300+,"Child-murderer and rapist, known as ""The Monst...",1969,1980
2,Javed Iqbal,Pakistan,100,100,"Child-murderer and rapist, known as ""Kukri"", I...",1998,1999
3,Mikhail Popkov,Russia,83,86+,"Serial rapist-killer nicknamed ""The Werewolf"",...",1992,2010
4,Daniel Camargo Barbosa,Colombia,72,180[14],"Child and woman murderer, believed to have pos...",1974,1986
5,Pedro Rodrigues Filho,Brazil,71,100+,Rodrigues Filho claimed to have killed over 10...,1967,2003
6,Kampatimar Shankariya,India,70,70+,Shankariya was an Indian serial killer who use...,1977,1978
7,Hoshang Amini,Iran,67,67,"Known as the ""Ghost of the Qanat Wells"". Murde...",1954,1962
8,Yang Xinhai,China,67,67,"Known as the ""Monster Killer"". Yang would ente...",1999,2003
9,Abul Djabar,Afghanistan,65,300+,Killed 65 men and boys by strangling them with...,1965,1970


## Duration Column

In [48]:
df['duration'] = df['End'].astype(int) - df['Start'].astype(int)

## Possible Victims
For the sake of simplicty, I'm going to use the largest number presented.

In [49]:
# remove bracketed numbers
df['Possible victims'] = df['Possible victims'].str.replace(r'\[\d+\]', '', regex=True)


def extract_largest_number(possible_victims):
    if pd.isna(possible_victims):
        return 0  # Convert NaN to 0
    elif isinstance(possible_victims, str):
        # Find all numbers in the string
        numbers = re.findall(r'\d+', possible_victims)
        # Convert numbers to integers
        numbers = list(map(int, numbers))
        # Return the largest number
        return max(numbers) if numbers else 0
    elif isinstance(possible_victims, (int, float)):
        return int(possible_victims)
    else:
        return 0


# Apply the function to the 'Possible Victims' column
df['Possible victims'] = df['Possible victims'].apply(extract_largest_number)



# remove + sign
# df['Possible victims'] = df['Possible victims'].str.replace('+', '', regex=True)

# # use larget number in range
# df['Possible victims'] = df['Possible victims'].apply(lambda x: max(map(int, re.findall(r'\d+', x))))

display(df.head(25))

,Name,Country,Proven victims,Possible victims,Notes,Start,End,duration
0,Luis Garavito,Colombia,193,300,"Child-murderer, torture-killer, and rapist kno...",1992,1999,7
1,Pedro López,Colombia,110,300,"Child-murderer and rapist, known as ""The Monst...",1969,1980,11
2,Javed Iqbal,Pakistan,100,100,"Child-murderer and rapist, known as ""Kukri"", I...",1998,1999,1
3,Mikhail Popkov,Russia,83,86,"Serial rapist-killer nicknamed ""The Werewolf"",...",1992,2010,18
4,Daniel Camargo Barbosa,Colombia,72,180,"Child and woman murderer, believed to have pos...",1974,1986,12
5,Pedro Rodrigues Filho,Brazil,71,100,Rodrigues Filho claimed to have killed over 10...,1967,2003,36
6,Kampatimar Shankariya,India,70,70,Shankariya was an Indian serial killer who use...,1977,1978,1
7,Hoshang Amini,Iran,67,67,"Known as the ""Ghost of the Qanat Wells"". Murde...",1954,1962,8
8,Yang Xinhai,China,67,67,"Known as the ""Monster Killer"". Yang would ente...",1999,2003,4
9,Abul Djabar,Afghanistan,65,300,Killed 65 men and boys by strangling them with...,1965,1970,5


# Append Geo Codes
Using the open source `Rest Countries` API, I will produce Lat/Lon for each of the countries in the dataset.

To be mindful of rate limits, I will break up the data, and run a distinct set of countries through the API, then join that data back to the original dataframe.  Some of these are not correct, so I'll work on them some more another time.

In [51]:
# Step 1: Extract unique country names from the DataFrame
unique_countries = df['Country'].unique()

# Step 2: Create a list to store country and its coordinates
country_coords = []

# Fetch lat/lon for each country
for country in unique_countries:
    lat, lon = get_lat_lon(country)
    country_coords.append({'Country': country, 'Latitude': lat, 'Longitude': lon})

# Convert the list to a DataFrame
coords_df = pd.DataFrame(country_coords)

# Step 4: Perform a left join to merge with the original DataFrame
df = df.merge(coords_df, on='Country', how='left')

# Display the updated DataFrame
display(df.head())


,Name,Country,Proven victims,Possible victims,Notes,Start,End,duration,Latitude,Longitude
0,Luis Garavito,Colombia,193,300,"Child-murderer, torture-killer, and rapist kno...",1992,1999,7,4.0,-72.0
1,Pedro López,Colombia,110,300,"Child-murderer and rapist, known as ""The Monst...",1969,1980,11,4.0,-72.0
2,Javed Iqbal,Pakistan,100,100,"Child-murderer and rapist, known as ""Kukri"", I...",1998,1999,1,30.0,70.0
3,Mikhail Popkov,Russia,83,86,"Serial rapist-killer nicknamed ""The Werewolf"",...",1992,2010,18,60.0,100.0
4,Daniel Camargo Barbosa,Colombia,72,180,"Child and woman murderer, believed to have pos...",1974,1986,12,4.0,-72.0


# Final Clean up

In [56]:
# Clean up the 'Proven victims' column
df['Proven victims'] = df['Proven victims'].apply(extract_largest_number)

# Convert data types
df['Proven victims'] = df['Proven victims'].astype(int)
df['Start'] = df['Start'].astype(int)
df['End'] = df['End'].astype(int)


# Save Data

In [58]:
df.to_csv('data/serial_killers_wiki_cleaned.csv', index=False)